In [1]:
# Setup CSS Styles
from styles.plnbstyles import style_nb
style_nb("./styles/styles.css")

<div id="ps-header">
    <a href="https://developers.planet.com/" target="_blank">
<img src="https://www.planet.com/assets/logos/logo.svg" style="display:inline-block; margin-right: 10px;" width="100"/> <span style="font-size:18px">Developer Resource Center</span>
</a>

###  Planet Analytics API Tutorial

<h1 style="margin-top:10px;">Summary Statistics: Ships and Clouds</h1>
</div>
<div class="content-block">
    
## Overview
    
1. [Introduction](#1.-Introduction)
2. [Post a stats job request](#2.-Post-a-stats-job-request)
3. [Poll the stats job endpoint](#3.-Poll-the-stats-job-endpoint)
4. [Get the job report results](#4.-Get-the-job-report-results)
5. [Restructure the results into a pandas dataframe](#5.-Restructure-the-results-into-a-pandas-dataframe)
6. [Visualize the time series](#6.-Visualize-the-time-series)


## 1. Introduction

This notebook demonstrates how to request ship and cloud summary statistics for a subscription using the Anaytics Feeds Stats API and visualize them as time series, enabling further analyses including patterns of life, development trends and anomaly detection.

The workflow involves:
- Posting a stats job request
- Polling the job stats endpoint
- Getting the job report results
- Restructuring the results into a pandas dataframe
- Visualizing the time series

## 2. Post a stats job request

### a) Define your subscription

In [5]:
ships_subscription_id='d493559c-2d9b-47e7-bd35-c04351544102'

### b) Visualize Subscription Bounds

In [39]:
import os
import requests
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

def jpp(data):
    print(json.dumps(data, indent=4))

In [40]:
# Get Subscription Geometry from Feeds API
API_KEY = os.environ['PL_API_KEY']
BASIC_AUTH = (API_KEY, '')
session = requests.Session()
session.auth=BASIC_AUTH

SIF_BASE_URL = 'https://sif-live.prod.planet-labs.com/'
subscription_get_url = SIF_BASE_URL + f'subscriptions/{ships_subscription_id}'

subs_get_resp = requests.get(
    subscription_get_url, 
    auth=BASIC_AUTH
)

# Convert geometry to a Geopandas DataFrame
subscription_geometry = subs_get_resp.json()['geometry']
subscription_gdf = gpd.GeoDataFrame.from_file(json.dumps(subs_get_resp.json()))

# Visualize Geometry with KeplerGL
subscription_map = KeplerGl(height=500)
subscription_map.add_data(data=subscription_gdf, name=f'Bandar Abbas Subscription {ships_subscription_id}')
subscription_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Bandar Abbas Subscription d493559c-2d9b-47e7-bd35-c04351544102': {'index': [0], 'columns': ['g…

### c) Post a stats report job request to the AF API

In [49]:
import os
import requests
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

def jpp(data):
    print(json.dumps(data, indent=4))

API_KEY = os.environ['PL_API_KEY']
BASIC_AUTH = (API_KEY, '')
session = requests.Session()
session.auth=BASIC_AUTH

request_body = {
    "title": "Bandar Abbas Cloud Contextualized Ship Stats March 2020",
    "subscriptionID": ships_subscription_id,
    "interval": "day",
    "startTime": "2020-03-01T00:00:00Z",
    "endTime": "2020-04-01T00:00:00Z"
}

SIF_BASE_URL = 'https://sif-live.prod.planet-labs.com/'
stats_post_url = SIF_BASE_URL + 'stats'

job_post_resp = requests.post(
    stats_post_url, 
    data=json.dumps(request_body), 
    auth=BASIC_AUTH,
    headers={'content-type':'application/json'}
)

jpp(job_post_resp.json())

{
    "created": "2020-04-27T22:01:44.057Z",
    "id": "f3c8d0c9-d6b1-443c-9cb4-dd6336f9df60",
    "links": [
        {
            "href": "https://sif-live.prod.planet-labs.com/stats/f3c8d0c9-d6b1-443c-9cb4-dd6336f9df60",
            "rel": "self"
        },
        {
            "href": "https://sif-live.prod.planet-labs.com/subscriptions/d493559c-2d9b-47e7-bd35-c04351544102",
            "rel": "subscription"
        }
    ],
    "status": "pending",
    "subscriptionID": "d493559c-2d9b-47e7-bd35-c04351544102",
    "title": "Bandar Abbas Cloud Contextualized Ship Stats March 2020",
    "updated": "2020-04-27T22:01:44.057Z"
}


## 3. Poll the stats job endpoint

In [50]:
import time

job_link = job_post_resp.json()['links'][0]['href']
status = "pending"
while status != "completed":
    report_status_resp = requests.get(
        job_link,
        auth=BASIC_AUTH,
    )
    status = report_status_resp.json()['status']
    print(status)
    time.sleep(2)
    
    
jpp(report_status_resp.json())

pending
pending
pending
pending
pending
pending
pending
pending
pending
pending
started
started
started
started
started
completed
{
    "created": "2020-04-27T22:01:44.057Z",
    "id": "f3c8d0c9-d6b1-443c-9cb4-dd6336f9df60",
    "links": [
        {
            "href": "https://sif-live.prod.planet-labs.com/stats/f3c8d0c9-d6b1-443c-9cb4-dd6336f9df60",
            "rel": "self"
        },
        {
            "href": "https://sif-live.prod.planet-labs.com/subscriptions/d493559c-2d9b-47e7-bd35-c04351544102",
            "rel": "subscription"
        },
        {
            "href": "https://sif-live.prod.planet-labs.com/stats/f3c8d0c9-d6b1-443c-9cb4-dd6336f9df60/report",
            "rel": "report"
        }
    ],
    "status": "completed",
    "subscriptionID": "d493559c-2d9b-47e7-bd35-c04351544102",
    "title": "Bandar Abbas Cloud Contextualized Ship Stats March 2020",
    "updated": "2020-04-27T22:02:24.938Z"
}


## 4. Get the job report results

In [51]:
report_results_link = report_status_resp.json()['links'][-1]['href']
report_results_link

'https://sif-live.prod.planet-labs.com/stats/f3c8d0c9-d6b1-443c-9cb4-dd6336f9df60/report'

In [52]:
results_resp = requests.get(
    report_results_link,
    auth=BASIC_AUTH,
)
print(results_resp.status_code)

200


## 5. Restructure the results into a pandas dataframe

In [53]:
import pandas as pd
from collections import defaultdict

def restructure_results(results_json):
    cols = results_json['cols']
    rows = results_json['rows']
    
    records = []
    for r in rows:
        rec = defaultdict()
        for i, cell in enumerate(r):
            rec[cols[i]['label']] = cell
        records.append(rec)
        
    df = pd.DataFrame.from_records(records)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df = df.set_index('Start Time')
    return df

In [54]:
df = restructure_results(results_resp.json())


In [55]:
df.describe()

,Total Object Count:0,Deduplicated Object Count:0,Clear Pixel Count (PSScene3Band-udm-clear_px):0,Total Pixel Count (PSScene3Band-udm-total_px):0,Clear Pixel Count (PSScene4Band-udm2-band_1):0,Total Pixel Count (PSScene4Band-udm2-total_px):0
count,32.000000,32.00000,32.0,32.0,32.0,32.0
mean,131.625000,119.68750,0.0,0.0,0.0,0.0
std,143.938103,126.99338,0.0,0.0,0.0,0.0
min,0.000000,0.00000,0.0,0.0,0.0,0.0
25%,9.750000,9.75000,0.0,0.0,0.0,0.0
50%,86.500000,85.00000,0.0,0.0,0.0,0.0
75%,198.250000,193.50000,0.0,0.0,0.0,0.0
max,445.000000,416.00000,0.0,0.0,0.0,0.0
